In [1]:
import nltk
import re
import pandas as pd
from nltk.corpus import stopwords
from tqdm.autonotebook import tqdm, trange
from natasha import AddressExtractor
from deeppavlov import configs, build_model

/home/caym/Programs/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
data = pd.read_csv('data/post_data.csv')
data = data.dropna()
data = data.reset_index()
data.drop(['index'], axis=1, inplace=True)

In [3]:
df = data[:]

## Using deepPavlov

In [5]:
ner_model = build_model(configs.ner.ner_rus_bert, download=True)

2020-01-21 17:08:32.773 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz to /home/caym/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz
100%|██████████| 666M/666M [08:21<00:00, 1.33MB/s] 
2020-01-21 17:16:54.205 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /home/caym/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz archive into /home/caym/.deeppavlov/downloads/bert_models
2020-01-21 17:16:59.206 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz to /home/caym/.deeppavlov/ner_rus_bert_v1.tar.gz
100%|██████████| 1.32G/1.32G [19:01<00:00, 1.15MB/s]
2020-01-21 17:36:01.168 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /home/caym/.deeppavlov/ner_rus_bert_v1.tar.gz archive into /home/caym/.deeppavlov/models
2020-01-21 17

Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Ins

2020-01-21 17:36:28.84 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/caym/.deeppavlov/models/ner_rus_bert/model]



INFO:tensorflow:Restoring parameters from /home/caym/.deeppavlov/models/ner_rus_bert/model


In [8]:
ner_model(['Немного затруднён въезд на Литейный мост со стороны Воскресенской набережной, там Опелю было мало ширины и он прижался к БМВ, Кроссоверы перебираются через поребрик'])

[[['Немного',
   'затруднён',
   'въезд',
   'на',
   'Литейный',
   'мост',
   'со',
   'стороны',
   'Воскресенской',
   'набережной',
   ',',
   'там',
   'Опелю',
   'было',
   'мало',
   'ширины',
   'и',
   'он',
   'прижался',
   'к',
   'БМВ',
   ',',
   'Кроссоверы',
   'перебираются',
   'через',
   'поребрик']],
 [['O',
   'O',
   'O',
   'O',
   'B-LOC',
   'I-LOC',
   'O',
   'O',
   'B-LOC',
   'I-LOC',
   'O',
   'O',
   'B-PER',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']]]

In [10]:
f = open('data/pavlov.txt', 'w')
for i in tqdm(list(df['text'])):
    s = ''
    tok = i.split(' ')
    try:
        if len(tok) < 100:
            res = ner_model([i])
            for j in range(len(res[1][0])):
                if res[1][0][j] == 'B-LOC':
                    s += res[0][0][j] + ' '
            f.write(s + '\n')
        else:
            res = ner_model([" ".join(tok[:100])])
            for j in range(len(res[1][0])):
                if res[1][0][j] == 'B-LOC':
                    s += res[0][0][j] + ' '
            f.write(s + '\n')
    except:
        f.write('\n')
f.close()

In [8]:
pavlov = []
f = open('data/pavlov.txt', 'r')
pavlov = [line.strip() for line in f]

In [9]:
data['pavlov'] = pavlov

In [10]:
data.to_csv('data/new_with_street.csv', index=None)

# Пробовали использовать Yargy/Natasha, но хороших результатов не получилось

In [ ]:
morph = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words("russian")

def process_text(text):
    tok = text.split(' ')
    pattern = re.compile("[А-я+0-9-]+")
    words = pattern.findall(" ".join(tok))
    tokens = [word for word in words if word not in stop_words]
    return " ".join(tokens)

In [54]:
extractor = AddressExtractor()
sample = process_text(text)
matches = extractor(text)
for match in matches:
    print(match.fact)

Address(parts=[Settlement(name='Санкт-Петербург', type=None), Street(name='Планерная', type='улица'), Building(number='5', type=None)])


In [37]:
df['text'][8]

'1. Эвакуировали 384 лицей. (в 14:30 всех запустили назад) \n2. Эвакуировали гимназию 405\n3. В 14:00 эвакуировали 98 школу\n4. Эвакуировали гимназию 399\n5. Эвакуировали гимназию 41\n6. 295 гимназию эвакуируют.\n\nЭвакуировали учащихся из школы № 553 Фрунзенского района на улице Ярослава Гашека, 4, корп. 4'

In [38]:
k = 0
f = open('data/street_from_natasha.txt', 'w')
extractor = AddressExtractor()
for i in tqdm(list(df['text'])):
    sample = process_text(i)
    matches = extractor(sample)
    s = ''
    for match in matches:
        for j in range(0, len(match.fact.parts)):
            try:
                if match.fact.parts[j].type is None:
                    try:
                        s += match.fact.parts[j].number + ' '
                        continue
                    except:
                        s += match.fact.parts[j].name + ' '
                        continue
                try:
                    if match.fact.parts[j].number is None:
                        break
                    s += match.fact.parts[j].type + ' ' + match.fact.parts[j].number + ' '
                except:
                    if match.fact.parts[j].name is None:
                        continue
                s += match.fact.parts[j].type + ' ' + match.fact.parts[j].name + ' '
            except:
                continue
    f.write(s + '\n')
    if len(matches)>0:
        k += 1
f.close()